# Outlier detection
### using Dask, a library for task scheduling and blocked algorithms for parallel processing

In [3]:
# https://github.com/fnielsen/everything
from everything import *

In [45]:
import dask.dataframe as dd # mimics pandas
from dask import delayed

ImportError: cannot import name 'delayed'

In [30]:
# Read dataframe with features for companies
filename = expanduser('~/CVR/virksomheder-features.csv')
#df = dd.read_csv(filename, encoding='utf-8', index_col=0)
#ValueError: 
#        The index column cannot be set at dataframe creation time. Instead use
#        the `set_index` method on the dataframe after it is created.
df = dd.read_csv(filename, encoding='utf-8')

ValueError: 
        The index column cannot be set at dataframe creation time. Instead use
        the `set_index` method on the dataframe after it is created.
        

In [7]:
df.columns

Index(['cvr_nummer', 'antal_penheder', 'branche_ansvarskode',
       'nyeste_antal_ansatte', 'nyeste_virksomhedsform', 'reklamebeskyttet',
       'sammensat_status', 'sidste_virksomhedsstatus', 'stiftelsesaar'],
      dtype='object')

In [29]:
df.set_index('cvr_nummer')

dd.DataFrame<set-partition--collect-a939158558c9ca6d0c51c4af4793b521, divisions=("b'10000009'", "b'21534196'", "b'28487436'", "b'32223737'", "b'35895752'", "b'99998059'")>

In [9]:
df.head()

,cvr_nummer,antal_penheder,branche_ansvarskode,nyeste_antal_ansatte,nyeste_virksomhedsform,reklamebeskyttet,sammensat_status,sidste_virksomhedsstatus,stiftelsesaar
0,b'33628234',b'1',b'None',b'0',b'Interessentskab',b'False',b'Oph\xc3\xb8rt',b'NORMAL',b'2011'
1,b'33627424',b'1',b'None',b'0',b'Enkeltmandsvirksomhed',b'False',b'Oph\xc3\xb8rt',b'NORMAL',b'2011'
2,b'33628404',b'1',b'None',b'0',b'Enkeltmandsvirksomhed',b'False',b'Oph\xc3\xb8rt',b'NORMAL',b'2011'
3,b'33631634',b'1',b'None',b'0',b'Enkeltmandsvirksomhed',b'False',b'Oph\xc3\xb8rt',b'NORMAL',b'2011'
4,b'33629621',b'1',b'None',b'0',b'Enkeltmandsvirksomhed',b'True',b'Oph\xc3\xb8rt',b'NORMAL',b'2011'


In [17]:
df.dtypes

cvr_nummer                  object
antal_penheder              object
branche_ansvarskode         object
nyeste_antal_ansatte        object
nyeste_virksomhedsform      object
reklamebeskyttet            object
sammensat_status            object
sidste_virksomhedsstatus    object
stiftelsesaar               object
dtype: object

In [39]:
df.where(lambda entry: entry == df[df.columns[1]].unique()).compute()

ValueError: where requires an ndarray like object for its condition

Traceback
---------
  File "/home/folzd/anaconda3/lib/python3.5/site-packages/dask/async.py", line 264, in execute_task
    result = _execute_task(task, data)
  File "/home/folzd/anaconda3/lib/python3.5/site-packages/dask/async.py", line 245, in _execute_task
    args2 = [_execute_task(a, cache) for a in args]
  File "/home/folzd/anaconda3/lib/python3.5/site-packages/dask/async.py", line 245, in <listcomp>
    args2 = [_execute_task(a, cache) for a in args]
  File "/home/folzd/anaconda3/lib/python3.5/site-packages/dask/async.py", line 246, in _execute_task
    return func(*args2)
  File "/home/folzd/anaconda3/lib/python3.5/site-packages/dask/compatibility.py", line 28, in apply
    return func(*args)
  File "/home/folzd/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py", line 4288, in where
    raise ValueError('where requires an ndarray like object for '


In [35]:
# one-hot-encoding df
dfn = df.map_partitions(lambda df: df.assign(z=df.x * df.y))
(df[df.columns[1]].unique()).compute()

0         b'1'
1         b'2'
2         b'0'
3         b'3'
4        b'13'
5         b'4'
6         b'5'
7         b'7'
8         b'8'
9        b'19'
10        b'6'
11       b'12'
12       b'16'
13       b'23'
14       b'21'
15      b'147'
16      b'119'
17      b'138'
18      b'385'
19       b'10'
20       b'11'
21       b'14'
22       b'44'
23       b'38'
24      b'130'
25      b'135'
26       b'88'
27       b'31'
28      b'131'
29       b'30'
        ...   
143     b'214'
144     b'126'
145     b'230'
146     b'100'
147     b'172'
148     b'250'
149     b'602'
150     b'103'
151     b'381'
152     b'161'
153     b'211'
154    b'1323'
155     b'132'
156     b'231'
157     b'137'
158      b'82'
159     b'283'
160     b'232'
161     b'121'
162     b'144'
163     b'158'
164     b'171'
165      b'76'
166     b'219'
167      b'79'
168     b'143'
169     b'163'
170     b'142'
171      b'81'
172     b'201'
Name: antal_penheder, dtype: object

In [23]:
tmp = df['cvr_nummer'].str.get_dummies()

In [5]:
# Functions for conversion to numerical dataframes
def to_dummies(df, column):
    datatype = df[column].dtypes
    if datatype in [int64, float64]:
        return df[[column]]
    elif datatype == bool:
        return df[[column]].astype(int)
    elif datatype == 'object':
        df_column = df[column].str.get_dummies()
        df_column.columns = [column + ":" + col for col in df_column.columns]
        return df_column
    else:
        raise ValueError('Unrecognized datatype for column {}'.format(column))
        
def dataframe_to_numerical(df):
    df_numerical = DataFrame(index=df.index)
    for column in df.columns:
        print(column)
        df_numerical = df_numerical.join(to_dummies(df, column))
    return df_numerical

In [40]:
def one_hot_encode_series(s, categories, dtype='f8'):
    """Transform a pandas.Series into a sparse matrix by one-hot-encoding
    for the given `categories`"""
    cat = pd.Categorical(s, np.asarray(categories))
    codes = cat.codes
    n_features = len(cat.categories)
    n_samples = codes.size
    mask = codes != -1
    if np.any(~mask):
        raise ValueError("unknown categorical features present %s "
                        "during transform." % np.unique(s[~mask]))
    row_indices = np.arange(n_samples, dtype=np.int32)
    col_indices = codes
    data = np.ones(row_indices.size)
    return sparse.coo_matrix((data, (row_indices, col_indices)),
                            shape=(n_samples, n_features),
                            dtype=dtype).tocsr()


@delayed(pure=True)
def one_hot_encode(df, categories, dtype='f8'):
    """One-hot-encode a pandas.DataFrame.

    Parameters
    ----------
    df : pandas.DataFrame
    categories : dict
        A mapping of column name to an sequence of categories for the column.
    dtype : str, optional
        The dtype of the output array. Default is 'float64'.
    """
    arrs = [one_hot_encode_series(df[col], cats, dtype=dtype)
            for col, cats in sorted(categories.items())]
    return sparse.hstack(arrs).tocsr()

NameError: name 'delayed' is not defined

In [6]:
# Numerical dataframe
dfn = dataframe_to_numerical(df)
dfn.shape

antal_penheder
branche_ansvarskode
nyeste_antal_ansatte
nyeste_virksomhedsform
reklamebeskyttet
sammensat_status
sidste_virksomhedsstatus
stiftelsesaar


(1529578, 455)

In [7]:
dfn.describe()

,antal_penheder:b'0',antal_penheder:b'1',antal_penheder:b'10',antal_penheder:b'100',antal_penheder:b'101',antal_penheder:b'102',antal_penheder:b'103',antal_penheder:b'104',antal_penheder:b'105',antal_penheder:b'1060',...,stiftelsesaar:b'2008',stiftelsesaar:b'2009',stiftelsesaar:b'2010',stiftelsesaar:b'2011',stiftelsesaar:b'2012',stiftelsesaar:b'2013',stiftelsesaar:b'2014',stiftelsesaar:b'2015',stiftelsesaar:b'2016',stiftelsesaar:b'nan'
count,1.529578e+06,1.529578e+06,1.529578e+06,1.529578e+06,1.529578e+06,1.529578e+06,1.529578e+06,1.529578e+06,1.529578e+06,1.529578e+06,...,1.529578e+06,1.529578e+06,1.529578e+06,1.529578e+06,1.529578e+06,1.529578e+06,1.529578e+06,1.529578e+06,1.529578e+06,1.529578e+06
mean,4.770924e-02,9.370617e-01,5.883976e-05,6.537751e-07,6.537751e-07,1.961325e-06,1.307550e-06,1.961325e-06,1.961325e-06,6.537751e-07,...,4.308509e-02,3.526920e-02,3.725341e-02,3.733971e-02,3.634401e-02,3.765875e-02,4.334137e-02,4.639319e-02,1.924910e-02,2.617062e-03
std,2.131504e-01,2.428520e-01,7.670485e-03,8.085636e-04,8.085636e-04,1.400472e-03,1.143481e-03,1.400472e-03,1.400472e-03,8.085636e-04,...,2.030487e-01,1.844595e-01,1.893822e-01,1.895929e-01,1.871447e-01,1.903696e-01,2.036245e-01,2.103352e-01,1.373994e-01,5.109026e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [8]:
# Preprocessing
imputer = Imputer()
scaler = StandardScaler(with_mean=False)
dfni = scaler.fit_transform(imputer.fit_transform(dfn))

In [9]:
# Outlier detection/novelty detection with K-means clustering
clusterer = MiniBatchKMeans(n_clusters=8, verbose=True)
clusterer.fit(dfni)

Init 1/3 with method: k-means++
Inertia for init 1/3: 43773.420297
Init 2/3 with method: k-means++
Inertia for init 2/3: 44037.763142
Init 3/3 with method: k-means++
Inertia for init 3/3: 44222.845400
Minibatch iteration 1/1529600: mean batch inertia: 163.605122, ewa inertia: 163.605122 
Minibatch iteration 2/1529600: mean batch inertia: 170.754483, ewa inertia: 163.606057 
Minibatch iteration 3/1529600: mean batch inertia: 163.711190, ewa inertia: 163.606070 
Minibatch iteration 4/1529600: mean batch inertia: 108.809494, ewa inertia: 163.598905 
Minibatch iteration 5/1529600: mean batch inertia: 107.468062, ewa inertia: 163.591566 
Minibatch iteration 6/1529600: mean batch inertia: 105.915432, ewa inertia: 163.584025 
Minibatch iteration 7/1529600: mean batch inertia: 97.462883, ewa inertia: 163.575379 
Minibatch iteration 8/1529600: mean batch inertia: 180.193591, ewa inertia: 163.577552 
Minibatch iteration 9/1529600: mean batch inertia: 120.561702, ewa inertia: 163.571927 
[MiniBat

MiniBatchKMeans(batch_size=100, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=10, n_clusters=8,
        n_init=3, random_state=None, reassignment_ratio=0.01, tol=0.0,
        verbose=True)

In [10]:
distances = sum((dfni - clusterer.cluster_centers_[clusterer.labels_, :]) ** 2, axis=1)
indices_clusterer = argsort(-distances)

In [10]:
dfn.iloc[indices_clusterer[:20], :]

,antal_penheder,branche_ansvarskode:0,branche_ansvarskode:15,branche_ansvarskode:65,branche_ansvarskode:75,branche_ansvarskode:76,branche_ansvarskode:96,branche_ansvarskode:97,branche_ansvarskode:99,branche_ansvarskode:None,...,sidste_virksomhedsstatus:UNDER FRIVILLIG LIKVIDATION,sidste_virksomhedsstatus:UNDER KONKURS,sidste_virksomhedsstatus:UNDER REASSUMERING,sidste_virksomhedsstatus:UNDER REASSUMMERING,sidste_virksomhedsstatus:UNDER REASUMMATION,sidste_virksomhedsstatus:UNDER REASUMMERING,sidste_virksomhedsstatus:UNDER REKONSTRUKTION,sidste_virksomhedsstatus:UNDER TVANGSOPLØSNING,sidste_virksomhedsstatus:slettet,stiftelsesaar
cvr_nummer,,,,,,,,,,,,,,,,,,,,,
15706538,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1992.0
20899301,1,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,1998.0
21976415,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1999.0
35852492,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,2014.0
31086477,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,2007.0
36467223,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,2014.0
56799559,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1985.0
27255116,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,2003.0
29219508,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,2006.0


In [11]:
filename = expanduser('~/workspace/cvrminer/virksomheder-report.html')
with codecs.open(filename, 'w', encoding='utf-8') as f:
    f.write("""
<html>
  <head>
    <title>Virksomheder report</title>
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
  </head>
  <body>
    <h1>Virksomheder report</h1>
    
""")
    
    f.write("""<h2>Numerical features dataframe summary statistics</h2>""")
    f.write(dfn.describe().T.to_html())
    f.write("""<h2>Novelty from k-means</h2>""")
    f.write(dfn.iloc[indices_clusterer[:50], :].to_html(
            escape=False,
            formatters={'__index__': 
                        lambda idx: '<a href="http://datacvr.virk.dk/data/visenhed?enhedstype=virksomhed&id={}">{}</a>'.format(
                     idx, idx)}))
    f.write("""
  </body>
</html>""")

In [13]:
# Investigate cluster model as a function of number of clusters
inertias = []
for n_clusters in range(1, 50):
    clusterer = MiniBatchKMeans(n_clusters=n_clusters, max_iter=200, max_no_improvement=30, n_init=10)
    clusterer.fit(dfni)
    inertias.append(clusterer.inertia_)

In [16]:
plot(inertias)
show()

In [ ]:
help(OneClassSVM)

In [ ]:
# Outlier detection with One-class-SVM
one_class_svm = OneClassSVM()
one_class_svm.fit(dfni)

In [ ]:
decisions = one_class_svm.decision_function(dfni)

In [ ]:
indices = argsort(decisions, axis=0)[:, 0]

In [ ]:
dfn.iloc[indices[:30], :]

In [ ]:
help(dfn.iloc[indices_clusterer[:20], :].to_html)